# Adversarial Search: Solving Tic-Tac-Toe with Heuristic Alpha-Beta Tree Search

## Introduction 

Multiplayer games can be implemented as:
1. Nondeterministic actions: The opponent is seen as part of an environment with nondeterministic actions. Non-determinism is the result of the unknown opponent's moves. 
2. Optimal Decisions: Minimax search (search complete game tree) and alpha-beta pruning.
3. __Heuristic Alpha-Beta Tree Search:__ Cut off tree search and use heuristic to estimate state value. 
4. Monte Carlo Tree search: Simulate playouts to estimate state value. 

Here we will implement search for Tic-Tac-Toe (see [rules](https://en.wikipedia.org/wiki/Tic-tac-toe)). The game is a __zero-sum game__: Win by x results in +1, win by o in -1 and a tie has a value of 0. Max plays x and tries to maximize the outcome while Min plays o and tries to minimize the outcome.   

We will implement

TODO

The algorithms search the game tree and we could return a conditional plan (or partial plan if cut offs are used), but the implementation here only identifies and returns the optimal next move.

## State Space and Search Tree Size

Each state is a possible board. Each of the 9 squares can have 3 values (empty, x and o), but some boards are impossible (where a player has several sequences of 3).The number of states in the state space graph is less than:

In [1]:
3**9

19683

A search tree (called game tree) can be superimposed on the state space graph. Note that a state can be in several branches of the tree resulting in more notes. 

* The complete search tree has a maximal depth $m=9$.
* The max branching factor $b=9$ (for first move).

DFS has a time complecity of $O(b^m)$ and a space complexity of $O(bm)$.

In [2]:
9**9

387420489

The search tree has $9 \times 8 \times \dots \times 2 \times 1$ many terminal nodes. This is equivalent to the number of permutations of the sequence 1 through 9:

In [3]:
import math

math.factorial(9)

362880

The number of terminal nodes of the complete search tree are less than (some are not possible because a player won twice, etc. We can reach a complete board with that many sequences. Some sequences are cut short because of a win and therefore there are less terminal notes.

It takes 9 moves to get to a complete board. An upper bound on the tree sice is the number of terminal notes $\times$ max tree depth:

In [4]:
math.factorial(9) * 9

3265920

__Note:__ This size makes this a very small problem that can be easily solved by searching the complete game tree. Most games and real problems are to large and can only be.

## The board

I represent the board as a vector of length 9. The values are `' ', 'x', 'o'`.  

In [5]:
def empty_board():
    return [' '] * 9

board = empty_board()
display(board)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Some helper functions.

In [6]:
import numpy as np

def show_board(board):
    """display the board"""
    b = np.array(board).reshape((3,3))
    print(b)

board = empty_board()
show_board(board)    

print()
print("Add some x's")
board[0] = 'x'; board[3] = 'x'; board[6] = 'x';  
show_board(board)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Add some x's
[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


In [7]:
def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""
    
    board = np.array(board).reshape((3,3))
    
    diagonals = np.array([[board[i][i] for i in range(len(board))], 
                          [board[i][len(board)-i-1] for i in range(len(board))]])
    
    for a_board in [board, np.transpose(board), diagonals]:
        for row in a_board:
            if len(set(row)) == 1 and row[0] != ' ':
                return row[0]
    
    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'

show_board(board)
print('Win? ' + check_win(board))

print()
show_board(empty_board())
print('Win? ' + check_win(empty_board()))

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
Win? x

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Win? n


In [8]:
def get_actions(board):
    """return possible actions as a vector ot indices"""
    return np.where(np.array(board) == ' ')[0].tolist()

    # randomize the action order
    #actions = np.where(np.array(board) == ' ')[0]
    #np.random.shuffle(actions)
    #return actions.tolist()


show_board(board)
get_actions(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


[1, 2, 4, 5, 7, 8]

# Heuristic Alpha-Beta Tree Search

See AIMA page 156ff. 


In [9]:
def result(state, player, action):
    """Add move to the board."""
    
    state = state.copy()
    state[action] = player
  
    return state

show_board(empty_board())

print()
print("State for placing an x at position 4:")
show_board(result(empty_board(), 'x', 4))

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

State for placing an x at position 4:
[[' ' ' ' ' ']
 [' ' 'x' ' ']
 [' ' ' ' ' ']]


Return utility for a state. Terminal states return the utility for Max as `+1`, -`1` or `0`. Non-terminal state have no utility and return `None`. Note that I use `utility is not None` to identify terminal states. That is, I use for `is_terminal(s)`.

In [10]:
def utility(state):
    """check is a state is terminal and return the utility if it is. None means not a terminal mode."""
    goal = check_win(state)        
    if goal == 'x': return +1 
    if goal == 'd': return 0  
    if goal == 'o': return -1  # loss is failure
    return None # continue

print(utility(['x'] * 9))
print(utility(['o'] * 9))
print(utility(empty_board()))

1
-1
None


In [11]:
def eval_fun(state):
    """heuristic for utiliy of state. Returns score and if it is a terminal state.
    1. For terminal states it is the utility. 
    2. For non-terminal state it uses a weighted linear function using features of the state. 
    The features we look at are 2 in a row/col/diagonal where the 3rd square is empty.
    We also scale wins and losses to 10, -10 to make sure the heuristic value stays in that range."""
    
    # terminal state?
    u = utility(state)
    if u is not None: return u * 10, True
    
    
    score = 0
    board = np.array(state).reshape((3,3))
    diagonals = np.array([[board[i][i] for i in range(len(board))], 
                          [board[i][len(board)-i-1] for i in range(len(board))]])
    
    for a_board in [board, np.transpose(board), diagonals]:
        for row in a_board:
            if sum(row == 'x') == 2 and any(row ==' '): score += 1
            if sum(row == 'o') == 2 and any(row ==' '): score -= 1
    
    return score, False

board = empty_board() 
show_board(board)
print(eval_fun(board))

board = empty_board() 
board[0] = 'x'
board[1] = 'x'
board[2] = 'x' 
show_board(board)
print(eval_fun(board))

board = empty_board() 
board[0] = 'x'
board[1] = 'x'
board[3] = 'x' 
board[4] = 'o'
board[8] = 'o'
show_board(board)
print(eval_fun(board))

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[3] = 'x' 
board[4] = 'o'
show_board(board)
print(eval_fun(board))

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
(0, False)
[['x' 'x' 'x']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
(10, True)
[['x' 'x' ' ']
 ['x' 'o' ' ']
 [' ' ' ' 'o']]
(2, False)
[['x' 'o' ' ']
 ['x' 'o' ' ']
 [' ' ' ' ' ']]
(0, False)


__Note:__ Starting with an empty board searched the complete game tree. The number of nodes above is the actual size of the complete game tree.

# Recursive DFS algorithm for Minimax Search with Alpha-Beta Pruning and Cutoff

See AIMA page 156ff. 

We cut off minimax search with alpha-beta pruning at depth and use the heuristic evaluation function to determine the next move.

The implementation is for player 'x' returns a the next move.

In [12]:
# global variables
DEBUG = 1 # 1 ... count nodes, 2 ... debug each node
COUNT = 0

def alpha_beta_search(board, cutoff = None):
    """start the search."""
    global DEBUG, COUNT
    COUNT = 0
    
    value, move = max_value_ab(board, -math.inf, +math.inf, 0, cutoff)
    
    if DEBUG >= 1: print(f"Number of nodes searched (cutoff = {cutoff}): {COUNT}") 
    
    return value, move

def max_value_ab(state, alpha, beta, depth, cutoff):
    """player's best move."""
    global DEBUG, COUNT
    COUNT += 1
    
    # cut off and terminal test
    v, terminal = eval_fun(state)
    if((cutoff is not None and depth >= cutoff) or terminal): 
        if DEBUG >= 2: print(f"stopped at {depth}: {state} [{alpha}, {beta}] eval: {v}" ) 
        return v, None
    
    # check is a terminal state
    #v = utility(state)
    #if DEBUG >= 2: print("max: " + str(state) + str([alpha, beta, v]) ) 
    #if v is not None: return v, None
    
    v, move = -math.inf, None

    # check all possible actions in the state, update alpha and return move with the largest value
    for a in get_actions(state):
        v2, a2 = min_value_ab(result(state, 'x', a), alpha, beta, depth + 1, cutoff)
        if v2 > v:
            v, move = v2, a
            alpha = max(alpha, v)
        if v >= beta: return v, move
    
    return v, move

def min_value_ab(state, alpha, beta, depth, cutoff):
    """opponent's best response."""
    global DEBUG, COUNT
    COUNT += 1
    
    # cut off and terminal test
    v, terminal = eval_fun(state)
    #if((cutoff is not None and depth >= cutoff) or terminal): 
    # always let the opponent make her move
    if(terminal): 
        if DEBUG >= 2: print(f"stopped at {depth}: {state} [{alpha}, {beta}] eval: {v}" ) 
        return v, None
    
    # return utility of state is a terminal state
    #v = utility(state)
    #if DEBUG >= 2: print("min: " + str(state) + str([alpha, beta, v]) ) 
    #if v is not None: return v, None
    
    v, move = +math.inf, None

    # check all possible actions in the state, update beta and return move with the smallest value
    for a in get_actions(state):
        v2, a2 = max_value_ab(result(state, 'o', a), alpha, beta, depth + 1, cutoff)
        if v2 < v:
            v, move = v2, a
            beta = min(beta, v)
        if v <= alpha: return v, move
    
    return v, move

__Notes:__

* The code check the available actions in order and picks the first one with the largest/smallest value. There are many ties. We could randomize `get_action()`.

## Some Tests

In [13]:
# x is about to win (play 8)

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'

print("Board:")
show_board(board)

print()
display(alpha_beta_search(board, 2))
display(alpha_beta_search(board, 4))
display(alpha_beta_search(board))

Board:
[['x' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' ' ']]

Number of nodes searched (cutoff = 2): 13


(10, 8)

Number of nodes searched (cutoff = 4): 47


(10, 2)

Number of nodes searched (cutoff = None): 61


(10, 2)

In [14]:
# o is about to win

board = empty_board() 
board[0] = 'o'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'
board[8] = 'x'

print("Board:")
show_board(board)

print()
display(alpha_beta_search(board, 2))
display(alpha_beta_search(board))

Board:
[['o' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' 'x']]

Number of nodes searched (cutoff = 2): 11


(-10, 2)

Number of nodes searched (cutoff = None): 15


(-10, 2)

In [15]:
#### x can draw if it chooses 7.

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[2] = 'x'
board[4] = 'o'

print("Board:")
show_board(board)

print()
display(alpha_beta_search(board, 2))
display(alpha_beta_search(board, 4))
display(alpha_beta_search(board))

Board:
[['x' 'o' 'x']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]

Number of nodes searched (cutoff = 2): 21


(-1, 7)

Number of nodes searched (cutoff = 4): 81


(0, 7)

Number of nodes searched (cutoff = None): 101


(0, 7)

In [16]:
# o went first

board = empty_board() 
board[4] = 'o'

print("Board:")
show_board(board)


print()
display(alpha_beta_search(board, 2))
display(alpha_beta_search(board, 4))
display(alpha_beta_search(board))

Board:
[[' ' ' ' ' ']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]

Number of nodes searched (cutoff = 2): 24


(-1, 0)

Number of nodes searched (cutoff = 4): 220


(-1, 0)

Number of nodes searched (cutoff = None): 2316


(0, 0)

In [17]:
# Empty board: Only a draw an be guaranteed

board = empty_board() 

print("Board:")
show_board(board)


print()
display(alpha_beta_search(board, 2))
display(alpha_beta_search(board, 4))
display(alpha_beta_search(board))

Board:
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Number of nodes searched (cutoff = 2): 26


(0, 0)

Number of nodes searched (cutoff = 4): 541


(0, 4)

Number of nodes searched (cutoff = None): 18297


(0, 0)

In [18]:
# A bad situation

board = empty_board() 
board[0] = 'o'
board[2] = 'x'
board[8] = 'o'

print("Board:")
show_board(board)


print()
display(alpha_beta_search(board, 2))
display(alpha_beta_search(board, 4))
display(alpha_beta_search(board))

Board:
[['o' ' ' 'x']
 [' ' ' ' ' ']
 [' ' ' ' 'o']]

Number of nodes searched (cutoff = 2): 26


(-2, 4)

Number of nodes searched (cutoff = 4): 148


(-10, 1)

Number of nodes searched (cutoff = None): 238


(-10, 1)